# Creating SPMD parallelism using OpenMP **parallel** directive

From this part, we begin to introduce how to use OpenMP directives to write programs. We first introduce the most basic and most commonly used **parallel** directive.

## Semantics and Syntax

### **parallel** Directive
The **parallel** directive is used to mark a parallel region. When a thread encounters a parallel region, a group of threads is created to execute the parallel region.
The original thread that executed the serial part will be the primary thread of the new team. All threads in the team execute parallel regions together. After a team is created, the number of threads in the team remains constant for the duration of that parallel region.

> Primary thread is also known as the master thread

When a thread team is created, the primary thread will implicitly create as many tasks as the number of threads, each task is assigned and bounded to one thread.
When threads are all occupied, implicit tasks that have not been allocated will be suspended waiting for idle threads.

The following example from Chapter 1 shows how to use the parallel directive in C.

In [1]:
//%compiler: clang
//%cflags: -fopenmp

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int main(int argc, char *argv[]){
    #pragma omp parallel
    printf("%s\n", "Hello World");
    
    return 0;
}

Hello World
Hello World
Hello World
Hello World
Hello World
Hello World
Hello World
Hello World
Hello World
Hello World
Hello World
Hello World


This example prints *Hello World* 8 times, which means 8 threads are created by defualt. The default number of threads is determined by the computer hardware, 8 threads are created on the author's computer. 
The following example shows how to use the num_threads clause in the parallel directive to specify the number of threads to create.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int main(int argc, char *argv[]){
    #pragma omp parallel num_threads(4)
    printf("%s\n", "Hello World");
    
    return 0;
}

In this example, we use the **num_threads** clause to specify the use of 4 threads to execute the parallel region. When the master thread encounters OpenMP constructs, three threads are created, and together with these three threads, a thread group of 4 is formed. 
*Hello World* is printed four times, once per thread.

Through the above examples, it is not difficult to find that the syntax of parallel directive in C is:

```
#pragma omp parallel [clause[ [,] clause] ... ] new-line
    structured-block
```

And the syntax of **num_threads** clause is：

```
num_threads(integer-expression)
```

The next two examples show how to use **paralle** diretcive in Fortran, and they have exactly same meaning as the two examples in C above.

In [5]:
!!%compiler: gfortran
!!%cflags: -fopenmp

PROGRAM Parallel_Hello_World
USE OMP_LIB

!$OMP PARALLEL

  PRINT *, "Hello World"

!$OMP END PARALLEL

END

 Hello World
 Hello World
 Hello World
 Hello World
 Hello World
 Hello World
 Hello World
 Hello World
 Hello World
 Hello World
 Hello World
 Hello World


In [13]:
!!%compiler: gfortran
!!%cflags: -fopenmp

PROGRAM Parallel_Hello_World
USE OMP_LIB

!$OMP PARALLEL num_threads(4)

  PRINT *, "Hello World"

!$OMP END PARALLEL

END

 Hello World
 Hello World
 Hello World
 Hello World


The syntax of **parallel** directive in Fortran is:
```
!$omp parallel do [clause[ [,] clause] ... ]
    loop-nest
[!$omp end parallel do]
```

Within a parallel region, the thread number uniquely identifies each thread. A thread can obtain its own thread number by calling the **omp_get_thread_num** library routine.

The following example is a little more complicated. It shows how to use the **omp_get_thread_num** library routine, and shows how to use two other clauses, the **default** clause and the **private** clause. It assigns tasks to each thread explicitly.

In [67]:
//%compiler: clang
//%cflags: -fopenmp
//This example is from https://www.openmp.org/wp-content/uploads/openmp-examples-5.1.pdf. The size of array and output are changed

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void subdomain(float *x, int istart, int ipoints) {
    int i;
    for (i = 0; i < ipoints; i++)       
         x[istart+i] = istart+i;
}

void sub(float *x, int npoints) {
    int iam, nt, ipoints, istart;
    #pragma omp parallel default(shared) private(iam,nt,ipoints,istart)
    {
        iam = omp_get_thread_num();
        nt = omp_get_num_threads();
        ipoints = npoints / nt; /* size of partition */
        istart = iam * ipoints; /* starting array index */
        if (iam == nt-1) /* last thread may do more */
            ipoints = npoints - istart;
        subdomain(x, istart, ipoints);
    }
}

void print(float *x, int npoints) {
    for (int i = 0; i < npoints; i++) {
        printf("%.5f ", x[i]);
    }
}

int main() {
    float array[100];
    sub(array, 100);
    print(array, 100);
    return 0;
}

0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000 10.00000 11.00000 12.00000 13.00000 14.00000 15.00000 16.00000 17.00000 18.00000 19.00000 20.00000 21.00000 22.00000 23.00000 24.00000 25.00000 26.00000 27.00000 28.00000 29.00000 30.00000 31.00000 32.00000 33.00000 34.00000 35.00000 36.00000 37.00000 38.00000 39.00000 40.00000 41.00000 42.00000 43.00000 44.00000 45.00000 46.00000 47.00000 48.00000 49.00000 50.00000 51.00000 52.00000 53.00000 54.00000 55.00000 56.00000 57.00000 58.00000 59.00000 60.00000 61.00000 62.00000 63.00000 64.00000 65.00000 66.00000 67.00000 68.00000 69.00000 70.00000 71.00000 72.00000 73.00000 74.00000 75.00000 76.00000 77.00000 78.00000 79.00000 80.00000 81.00000 82.00000 83.00000 84.00000 85.00000 86.00000 87.00000 88.00000 89.00000 90.00000 91.00000 92.00000 93.00000 94.00000 95.00000 96.00000 97.00000 98.00000 99.00000 

In the above example, we use the default number of threads to perform assignment operations on 100 elements in the array. Tasks are evenly distributed to each thread, and when the number of tasks is not divisible by the number of threads, the remaining tasks will be completed by the last thread.

When programming in parallel, the most important and hardest part is how to assign tasks and manage threads. We already introduced that a thread can get its own id through the **omp_get_thread_num** routine. Another important routine is **omp_get_num_threads**, which returns the number of threads in the current team.

In the above example, variable *npoints* presents the total number of elements in the array, and it is divided into *nt* parts, each of size *ipoints*. The starting address of each part is *istart*. Each part is completed by one thread, and a total of 8 threads execute tasks in parallel.

The **default** clause is used to define the default data-sharing attributes of variables that are referenced in a parallel, teams, or task generating construct. In the above example, *default(shared)* indicates that by default, the variables in the parallel region are shared variables.
The **private** clause is used to explicitly specify variables that are private in each task or SIMD lane (SIMD will be introduced in the next chapter). In the above example, the variables *iam, nt, ipoints* and *istart* are private variables for each thread, which means a thread cannot access these variables of another thread.

Both of these two clauses belong to the data-sharing attribute clauses, which we will introduce in detail in the section of clauses later.

The corresponding Fortran program is shown below.

In [66]:
!!%compiler: gfortran
!!%cflags: -fopenmp

    SUBROUTINE SUBDOMAIN(X, ISTART, IPOINTS)
        INTEGER ISTART, IPOINTS
        REAL X(0:99)

        INTEGER I

        DO 100 I=0,IPOINTS-1
            X(ISTART+I) = ISTART+I
100     CONTINUE

    END SUBROUTINE SUBDOMAIN

    SUBROUTINE SUB(X, NPOINTS)
        INCLUDE "omp_lib.h" ! or USE OMP_LIB

        REAL X(0:99)
        INTEGER NPOINTS
        INTEGER IAM, NT, IPOINTS, ISTART

!$OMP PARALLEL DEFAULT(PRIVATE) SHARED(X,NPOINTS)

        IAM = OMP_GET_THREAD_NUM()
        NT = OMP_GET_NUM_THREADS()
        IPOINTS = NPOINTS/NT
        ISTART = IAM * IPOINTS
        IF (IAM .EQ. NT-1) THEN
            IPOINTS = NPOINTS - ISTART
        ENDIF
        CALL SUBDOMAIN(X,ISTART,IPOINTS)

!$OMP END PARALLEL
    END SUBROUTINE SUB
    
    SUBROUTINE print(X, NPOINTS)
        INTEGER I
        REAL X(0:99)
        INTEGER NPOINTS
        DO I = 0,NPOINTS-1
            WRITE(*,'(1x,f9.5,$)') X(I)
        END DO
    END SUBROUTINE PRINT

    PROGRAM PAREXAMPLE
        REAL ARRAY(100)
        CALL SUB(ARRAY, 100)
        CALL PRINT(ARRAY, 100)
    END PROGRAM PAREXAMPLE

   0.00000   1.00000   2.00000   3.00000   4.00000   5.00000   6.00000   7.00000   8.00000   9.00000  10.00000  11.00000  12.00000  13.00000  14.00000  15.00000  16.00000  17.00000  18.00000  19.00000  20.00000  21.00000  22.00000  23.00000  24.00000  25.00000  26.00000  27.00000  28.00000  29.00000  30.00000  31.00000  32.00000  33.00000  34.00000  35.00000  36.00000  37.00000  38.00000  39.00000  40.00000  41.00000  42.00000  43.00000  44.00000  45.00000  46.00000  47.00000  48.00000  49.00000  50.00000  51.00000  52.00000  53.00000  54.00000  55.00000  56.00000  57.00000  58.00000  59.00000  60.00000  61.00000  62.00000  63.00000  64.00000  65.00000  66.00000  67.00000  68.00000  69.00000  70.00000  71.00000  72.00000  73.00000  74.00000  75.00000  76.00000  77.00000  78.00000  79.00000  80.00000  81.00000  82.00000  83.00000  84.00000  85.00000  86.00000  87.00000  88.00000  89.00000  90.00000  91.00000  92.00000  93.00000  94.00000  95.00000  96.00000  97.00000  98.00000  99.00000

## Clauses

## Examples